# Spatial Operations
stough 202-

1. [Predictive Coding from spatial operations perspective](#predictive)
1. [Unsharp Masking](#unsharp) 
1. [Gradient Maps](#gradient)

To this point we have generally thought of images as collections of **independent** pixels, organized in a grid for sure but not really thought of as a connected collection. We have heavily considered the [histograms of images](../NumpyAndVisualization/matplotlib_tutorial.ipynb#Histograms) for use in [contrast enhancement](../Enhancement/enhance_histeq.ipynb) and [compression](../Entropy/entropy_intro.ipynb), where histogramming completely destroys the spatial relationships that these pixels have with one another. In considering entropy in fact, we modeled our image as a sequence of **independent and identically distributed (iid)** symbols sampled from the histogram (probability distribution).

We did hint at the importance of **spatial coherence**, which is exactly the way in which images are not iid sequences. We saw that **predictive coding**, using neighbors of a pixel to predict that pixel and then encoding only the error of prediction, leads to much more highly compressible signals than the images themselves. Were pixels iid, then such an encoding would be of no benefit. If our [power transform](../Enhancement/enhance_transfer.ipynb#power) is an example of a single-pixel operation (or point transform), then predictive coding is an example of a **spatial or neighborhood operation**.

**Spatial operations** are operations that we apply to the pixels of an image that take into account the neighborhood of a pixel. Linear versions of such operations are also called **spatial filters**. We're basically looking at the neighborhood of a pixel and computing a linear combination of the values/intensities in that neighborhood. 

Here we're going to look at some spatial operations using [`scipy.ndimage.correlate`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.correlate.html). 

## Imports
Bringing in [`scipy.ndimage.correlate`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.correlate.html) and [`convolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html). 

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

# For spatial filtering/operations
from scipy.ndimage import (correlate,
                           convolve)

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

from matrix_utils import (arr_info,
                          make_linmap)
from vis_utils import (vis_rgb_cube,
                       vis_hists,
                       vis_pair)

<a id='predictive'></a>

## Review Predictive Coding
Instead of directly using `np.diff`, we can use correlate. 

In [ ]:
I = plt.imread('../dip_pics/canyon.jpg')
vis_hists(I)
print(arr_info(I))

In [ ]:
J = I[...,0]
vis_hists(J)

In [ ]:
h = np.array([-1, 1], ndmin=2).astype('int16')

In [ ]:
h

In [ ]:
arr_info(h)

In [ ]:
Jf = correlate(J.astype('int16'), h, mode='constant', cval=0)
arr_info(Jf)

In [ ]:
vis_hists(Jf)

<a id='unsharp'></a>

## Unsharp Masking
Above we saw how to use a spatial filter to get at local pixel differences. 
Here we'll use a blur to actually sharpen the edges in an image.

In [ ]:
h = np.ones((5,5))/25.0
h

In [ ]:
Jf = correlate(J.astype('float'), h, mode='constant', cval=0)

In [ ]:
vis_pair(J, Jf, cmap='gray', second_title="Blurred")

In [ ]:
arr_info(Jf)

In [ ]:
arr_info(J)

In [ ]:
D = J - Jf
vis_hists(D)

In [ ]:
vis_pair(J, D, cmap='gray', second_title='Original - Blurred')

In [ ]:
Jsharp = J + 2*D 
# vis_pair(J, Jsharp, cmap='gray') # results in less contrast due to out-of-range pixels messing up display.
vis_pair(J, np.clip(Jsharp,0,255), cmap='gray', second_title="Sharpened")

In [ ]:
arr_info(Jsharp)

In [ ]:
plt.figure()
plt.plot(J[193, 250:270])
plt.plot(np.clip(Jsharp,0,255)[193, 250:270])

<a id='gradient'></a>

## Gradient Maps
An alternative filter design can give us locally computed vertical and horizontal *edginess* if you will. These first order derivative images in $x,y$ can allow us to compute the gradient map of an image. Basically the edginess of every pixel.

In [ ]:
I = plt.imread('../dip_pics/girl_with_glasses.png')
vis_hists(I)
print(arr_info(I))

In [ ]:
h = np.ones((5,5))/25.0
h

In [ ]:
h = np.array([[1, 0, -1],[2, 0, -2],[1, 0, -1]])

In [ ]:
h

This filter we've designed above responds to vertical edges; that is, to get a large magnitude response from correlation with a 3x3 image neighborhood, we would need the left side of this tiny neighborhood to look brighter than the right (or darker for a large negative response). This is the famous Sobel operator, which you can read more about [here](https://en.wikipedia.org/wiki/Sobel_operator) and [here](https://homepages.inf.ed.ac.uk/rbf/HIPR2/sobel.htm).

In [ ]:
plt.figure()
plt.imshow(h, cmap='gray')

In [ ]:
Gx = correlate(I[...,0], h, mode='nearest')

In [ ]:
vis_pair(I, Gx, cmap='gray', second_title="GX")

In [ ]:
Gy = correlate(I[...,0], h.transpose(), mode='nearest')

In [ ]:
vis_pair(Gx, Gy, cmap='gray', first_title='Gx', second_title='Gy')

The above are the first order derivatives in $x$ and $y$ for the original image when thought of as a height map, and using the Sobel operators to compute local difference. 

The gradient magnitude (how steep the height map is) is then the square root of the sum of the square dervatives. 

In [ ]:
G = np.sqrt(Gx**2 + Gy**2)
# G = Gx**2 + Gy**2

In [ ]:
arr_info(G)

In [ ]:
vis_pair(I, G.max() - G, cmap='gray', second_title='Grad Mag')

### Laplacian
second derivative

In [ ]:
h = np.array([[-1, -1, -1],[-1, 8, -1],[-1, -1, -1]])

In [ ]:
h

In [ ]:
J = correlate(I[...,0], h, mode='nearest')
arr_info(J)

In [ ]:
J = np.abs(J)
J = J/J.max()
arr_info(J)

In [ ]:
vis_pair(I, 1 - np.log2(J+1),  cmap='gray')